# 格式化生成

## 使用场景

- RAG 驱动的电商客服：当用户询问“推荐几款适合程序员的键盘”时，我们希望 LLM 返回一个包含产品名称、价格、特性和购买链接的 JSON 列表，而不是一段描述性文字，以便前端直接渲染成商品卡片。

- 自然语言转 API 调用：用户说“帮我查一下明天从上海到北京的航班”，系统需要将这句话解析成一个结构化的 API 请求，如 {"departure": "上海", "destination": "北京", "date": "2025-07-18"}。

- 数据自动提取：从一篇新闻文章中，自动抽取出事件、时间、地点、涉及人物等关键信息，并以结构化形式存入数据库。

在这些场景中，格式化生成是连接 LLM 的自然语言理解能力和下游应用程序的程序化逻辑之间的关键。

## 实现方法

### LangChain 中

LangChain 提供了一个强大的组件——OutputParsers（输出解析器），专门用于处理 LLM 的输出。

- 提供格式指令：在发送给 LLM 的提示（Prompt）中，自动注入一段关于如何格式化输出的指令。

- 解析模型输出：接收 LLM 返回的纯文本字符串，并将其解析成预期的结构化数据（如 Python 对象）。

详情见：https://docs.langchain.com/oss/python/langchain/structured-output

### LlamaIndex 中

LlamaIndex 的输出解析与生成过程紧密结合。

- 响应合成（Response Synthesis）：，检索器召回一系列相关的文本块（Nodes）后，响应合成器接收这些文本块和原始查询，并以一种更智能的方式将它们呈现给 LLM 以生成最终答案。

    - 它可以逐块处理信息并迭代地优化答案（refine 模式），或者将尽可能多的文本块压缩进单次 LLM 调用中（compact 模式）。这个阶段的默认目标是生成一段高质量的文本回答。

- 结构化输出（Structured Output）： 

    1. 定义 Schema：定义一个 Pydantic 模型，明确所需输出的数据结构、字段和类型。
   
    2. 引导生成：LlamaIndex 会将这个 Pydantic 模型转换成 LLM 能理解的格式指令。
   
    3. 解析验证：LM 返回的输出会被自动解析并用 Pydantic 模型进行验证，确保其类型和结构完全正确，最终返回一个 Pydantic 对象实例。

### 不依赖框架的简单实现思路

通过提示工程给出清晰、明确的指令和示例。

- 明确要求 JSON 格式：在提示中直接、强硬地要求模型“必须返回一个 JSON 对象”、“不要包含任何解释性文字，只返回 JSON”。

- 提供 JSON Schema：在提示中给出你想要的 JSON 对象的模式（Schema），描述每个键的含义和数据类型。

- 提供 few-shot 示例：给出 1-2 个“用户输入 -> 期望的 JSON 输出”的完整示例，让模型学习输出的格式和风格。

- 使用语法约束：对于一些本地部署的开源模型（如通过 llama.cpp 运行的模型），可以使用 GBNF (GGML BNF) 等语法文件来强制约束模型的输出，确保其生成的每一个 token 都严格符合预定义的 JSON 语法。**这是最严格也是最可靠的非 Function Calling 方法。**

## Function Calling

Function Calling（或称 Tool Calling）本质是一个多轮对话流程，让模型、代码和外部工具（如 API）协同工作。

相比于单纯通过提示工程“请求”模型输出 JSON，Function Calling 的优势在于：

- 可靠性更高：这是**模型原生支持的能力**，相比于解析可能格式不稳定的纯文本输出，这种方式得到的结构化数据更稳定、更精确。

- 意图识别：它不仅仅是格式化输出，更包含了“意图到函数的映射”。模型能根据用户问题主动选择最合适的工具。

- 与外部世界交互：它是构建能执行实际任务的 AI 代理（Agent）的核心基础，让 LLM 可以查询数据库、调用 API、控制智能家居等。

### 工作流程

1. 定义工具：首先，在代码中以特定格式（通常是 JSON Schema）定义好可用的工具，包括工具的名称、功能描述、以及需要的参数。

2. 用户提问：用户发起一个需要调用工具才能回答的请求。

3. 模型决策：模型接收到请求后，分析用户的意图，并匹配最合适的工具。

    - 它不会直接回答，而是返回一个包含 tool_calls 的特殊响应。
    - 这个响应相当于一个指令：“请调用某某工具，并使用这些参数”。

4. 代码执行：应用接收到这个指令，解析出工具名称和参数，然后在代码层面实际执行这个工具（例如，调用一个真实的天气 API）。

5. 结果反馈：将工具的执行结果（例如，从 API 获取的真实天气数据）包装成一个 role 为 tool 的消息，再次发送给模型。
   
6. 最终生成：模型接收到工具的执行结果后，结合原始问题和工具返回的信息，生成最终的、自然的语言回答。

In [ ]:
# 初始化大模型

from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com",
)


In [ ]:
# 定义一个函数，用于发送消息并获取模型的响应
def send_messages(messages, tools=None):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # 让模型自主决定是否调用工具
    )
    return response.choices[0].message


In [ ]:
# 定义工具
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "获取指定地点的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市和省份，例如：杭州市, 浙江省",
                    }
                },
                "required": ["location"]
            },
        }
    },
]


In [ ]:
# 用户提问，模型决策调用工具
messages = [{"role": "user", "content": "杭州今天天气怎么样？"}]
print(f"User> {messages[0]['content']}\n")
message = send_messages(messages, tools=tools)

# 执行工具，并将结果返回模型
if message.tool_calls:
    print("--- 模型发起了工具调用 ---")
    tool_call = message.tool_calls[0]
    function_info = tool_call.function
    print(f"工具名称: {function_info.name}")
    print(f"工具参数: {function_info.arguments}")

    # 将模型的回复（包含工具调用请求）添加到消息历史中
    messages.append(message)

    # 模拟执行工具
    tool_output = "24℃，晴朗"
    print(f"--- 执行工具并返回结果 ---")
    print(f"工具执行结果: {tool_output}\n")

    # 将工具的执行结果作为一个新的消息添加到历史中
    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": tool_output})

    # 第二次调用：将工具结果返回给模型，获取最终回答
    print("--- 将工具结果返回给模型，获取最终答案 ---")
    final_message = send_messages(messages, tools=tools)
    print(f"Model> {final_message.content}")
else:
    # 如果模型没有调用工具，直接打印其回答
    print(f"Model> {message.content}")


User> 杭州今天天气怎么样？

--- 模型发起了工具调用 ---
工具名称: get_weather
工具参数: {"location": "杭州市, 浙江省"}
--- 执行工具并返回结果 ---
工具执行结果: 24℃，晴朗

--- 将工具结果返回给模型，获取最终答案 ---
Model> 根据查询结果，杭州今天天气很好：

- **温度**：24℃
- **天气状况**：晴朗

今天是个适合外出活动的好天气！
